In [1]:
using LinearAlgebra

# Sampling Points from a Hyperplane Arrangement Complement

### Start with a random point

In [2]:
p(n) = [rand(Float64)-.5 for j=1:n]

p (generic function with 1 method)

### Then reflect over all hyperplanes in the arrangement

#### getH

Get all hyperplanes in the $B_n$ arrangement.

In [3]:
function getH(k, full=true)
    H = []
    if full
        for i=1:(2*k)
            v = zero(rand(2*k))
            v[i] = 1
            push!(H, v)
        end
        
        for i=1:(2*k)
            for j=(i+1):(2*k)
                v = zero(rand(2*k))
                v[i] = 1
                v[j] = -1
                push!(H, v)
            end
        end
    else
        v = zero(rand(2*k))
        v[1] = 1
        push!(H, v)
        for i=1:k
            for j=1:k
                v = zero(rand(2*k))
                v[i] = 1
                v[k + j] = -1
                push!(H, v)
            end
        end
    end
    return H
end

getH (generic function with 2 methods)

#### reflect

Reflect a vector $v$ over the hyperplane with normal vector $a$.

In [4]:
function reflect(v, a)
    return v .- a*2*(LinearAlgebra.dot(v,a))/LinearAlgebra.dot(a,a)
end

reflect (generic function with 1 method)

### Reflect over all hyperplanes

In [5]:
function getOrbit(v, H)
    k = Int(length(v)/2)
    orbit = [v]
    ineqs = [ineq(v, k, true)]
    to_check = [v]
    while length(to_check) > 0
        new_to_check = []
        for u in to_check
            for h in H
                r = reflect(u, h)
                I = ineq(r, k, true)
                if I ∉ ineqs
                    push!(orbit, r)
                    push!(new_to_check, r)
                    push!(ineqs, I)
                end
            end
        end
        to_check = new_to_check
    end
    return orbit
end

getOrbit (generic function with 1 method)

### Get inequalities

In [6]:
function ineq(v, k, full=false)
    if full
        return [[v[i] > 0 for i=1:(2*k)] [v[i] > v[j] for i=1:(2*k), j=1:(2*k)]]
    end
    top_row = [1 < 0 for i=1:k]
    top_row[1] = (v[1] > 0)
    return [top_row [v[i] > v[j] for i=1:k, j=(k+1):(2*k)] [v[i] > -v[j] for i=1:k, j=(k+1):(2*k)]]
end

ineq (generic function with 2 methods)

### Project to $\sum \eta_j = 0$

In [7]:
function project(v, a)
    return v - a*(LinearAlgebra.dot(v, a))/(LinearAlgebra.dot(a,a))
end

project (generic function with 1 method)

In [8]:
function getProj(orbit, k)
    z = zero(rand(2*k))
    for i=(k+1):(2*k)
        z[i] = 1
    end
    
    P = [project(u, z) for u in orbit]
    
    sample = []
    Is = []
    for u in P
        I = ineq(u, k)
        if I ∉ Is
            push!(sample, u)
            push!(Is, I)
        end
    end
    
    return sample
end

getProj (generic function with 1 method)

# Example (n = 3)

In [13]:
test_v = p(6);

In [14]:
H = getH(3);
length(H)

21

In [15]:
O = getOrbit(test_v, H);
print(length(O), '\n')

5040

In [16]:
sample = getProj(O, 3);
print(length(sample))

318

In [10]:
n = 3
test_v = p(2*n);
test_v[2*n] = sum(test_v[(n+1):(2*n-1)])
H = getH(n);
O = getOrbit(test_v, H);
print(length(O), '\n')
sample = getProj(O, n);
print(length(sample))

5040
335

In [11]:
z = zero(rand(2*n));
for i=(n+1):(2*n)
    z[i] = 1
end
P = [project(u, z) for u in O];
Is = [];
for i=eachindex(P)
    u = P[i]
    I = ineq(u, n)
    if I in Is
        print(i)
        break
    else
        push!(Is, I)
    end
end

7

In [12]:
Is = [ineq(u, n) for u in sample]

335-element Vector{Matrix{Bool}}:
 [0 0 1 0; 0 0 1 1; 0 0 1 0]
 [1 0 1 1; 0 0 1 1; 0 0 1 0]
 [0 0 1 0; 0 0 1 0; 0 0 1 0]
 [0 0 1 0; 0 0 1 1; 0 0 1 1]
 [0 0 1 0; 0 1 1 0; 0 0 0 0]
 [0 0 0 1; 0 0 0 1; 0 0 0 0]
 [1 0 1 1; 0 0 1 0; 0 0 1 0]
 [1 1 1 1; 0 0 1 0; 0 0 0 0]
 [0 0 0 0; 0 0 1 1; 0 0 0 0]
 [0 0 1 0; 0 0 1 0; 0 0 1 1]
 [0 0 1 0; 0 1 1 1; 0 0 0 0]
 [0 0 0 1; 0 0 0 0; 0 0 0 0]
 [0 0 1 0; 0 1 1 0; 0 1 1 1]
 ⋮
 [0 0 1 1; 0 1 1 1; 0 1 1 1]
 [1 1 0 0; 0 1 0 0; 0 1 1 0]
 [1 0 0 1; 0 1 1 1; 0 0 1 1]
 [1 1 0 0; 0 1 1 1; 0 1 0 1]
 [1 0 1 0; 0 1 1 1; 0 1 1 0]
 [1 1 0 0; 0 1 1 0; 0 1 1 1]
 [1 1 1 1; 0 1 0 0; 0 1 1 0]
 [1 1 1 1; 0 1 1 0; 0 1 0 0]
 [1 0 1 0; 0 0 0 0; 0 0 1 1]
 [1 1 0 0; 0 1 1 0; 0 1 1 0]
 [1 0 1 0; 0 1 1 1; 0 0 1 1]
 [1 1 0 0; 0 1 1 1; 0 1 1 0]

## Navigation between Weyl Chambers

In [9]:
function getNeighbors(orbit, v, k, full=false)
    return [u for u in orbit if isNeighbor(u, v, k, full)]
end

getNeighbors (generic function with 2 methods)

In [10]:
function isNeighbor(u, v, k, full=false)
    return sum(ineq(u, k, full) .⊻ ineq(v, k, full)) == 1
end

isNeighbor (generic function with 2 methods)

In [15]:
## test case
print(ineq([1,1,0,0], 2), '\n')
print(ineq([3,1,2,0], 2), '\n')
print(ineq([0,1,1,0], 2), '\n')
print(isNeighbor([1,1,0,0], [3,1,2,0], 2), '\n') ## true
print(isNeighbor([1,1,0,0], [1,1,0,0], 2), '\n') ## false
print(isNeighbor([1,1,0,0], [0,1,1,0], 2), '\n') ## false

Bool[1 1 1; 0 1 1]
Bool[1 1 1; 0 0 1]
Bool[0 0 0; 0 0 1]
true
false
false


In [16]:
## test cases for full
print(ineq([1,1,0,0], 2, true), '\n')
print(ineq([3,1,2,0], 2, true), '\n')
print(ineq([0,1,1,0], 2, true), '\n')
print(isNeighbor([1,1,0,0], [3,1,2,0], 2, true), '\n') ## false
print(isNeighbor([1,1,0,0], [1,1,0,0], 2, true), '\n') ## false
print(isNeighbor([1,1,0,0], [0,1,1,0], 2, true), '\n') ## false

Bool[1 0 0 1 1; 1 0 0 1 1; 0 0 0 0 0; 0 0 0 0 0]
Bool[1 0 1 1 1; 1 0 0 0 1; 1 0 1 0 1; 0 0 0 0 0]
Bool[0 0 0 0 0; 1 1 0 0 1; 1 1 0 0 1; 0 0 0 0 0]
false
false
false


In [13]:
include("lambda_cyclic.jl")

getRank (generic function with 1 method)

# Random Sampling

In [11]:
n=4
sample2 = []
for i=1:2^20
    v = p(2*n)
    v[2*n] = -sum(v[(n+1):(2*n-1)])
    push!(sample2, v)
end

Is2 = []
S2 = []
for i=eachindex(sample2)
    if i % 1000 == 0
        print(i)
    end
    v = sample2[i]
    I = ineq(v, n)
    if I ∉ Is2
        push!(S2, i)
        push!(Is2, I)
    end
end
length(Is2)

1000200030004000500060007000800090001000011000120001300014000150001600017000180001900020000210002200023000240002500026000270002800029000300003100032000330003400035000360003700038000390004000041000420004300044000450004600047000480004900050000510005200053000540005500056000570005800059000600006100062000630006400065000660006700068000690007000071000720007300074000750007600077000780007900080000810008200083000840008500086000870008800089000900009100092000930009400095000960009700098000990001000001010001020001030001040001050001060001070001080001090001100001110001120001130001140001150001160001170001180001190001200001210001220001230001240001250001260001270001280001290001300001310001320001330001340001350001360001370001380001390001400001410001420001430001440001450001460001470001480001490001500001510001520001530001540001550001560001570001580001590001600001610001620001630001640001650001660001670001680001690001700001710001720001730001740001750001760001770001780001790001800001810001820001830001840001850

208732

In [14]:
ranks = [getRank(s[1:n], s[(n+1):end], n) for s in sample2[S2]]
R = Set(ranks)

Set{Int64} with 7 elements:
  16
  14
  13
  15
  10
  12
  11

In [15]:
counts = [sum([1 for rank in ranks if rank == r]) for r in R]
print(counts, '\n')
dist = [c/length(ranks) for c in counts]

[10706, 85657, 45806, 44252, 937, 15496, 5878]


7-element Vector{Float64}:
 0.05129065021175479
 0.4103683191844087
 0.21944886265642066
 0.2120039093191269
 0.004489009830787804
 0.07423873675334879
 0.02816051204415231

In [16]:
for i=eachindex(ranks)
    r = ranks[i] ## get the rank of the current cell
    if r == minimum(ranks) ## for the minimal rank chambers
        N = [u for u=eachindex(S2) if isNeighbor(sample2[S2[u]], sample2[S2[i]], n)] ## get a list of neighboring samples
        nranks = ranks[N] ## get the ranks of neighboring cells
        if maximum(nranks) - r > 1 ## if the rank jumps by more than 1
            println(i, maximum(nranks) - r)
        end
    end
end

6389310131248133816921718194725012927337433753768427544175151521361487812816082418494910492409453969399101006910758113581209113371135721387814735149011493115204154081570816098170201736217538190101911519420196531985519983204172046520698209312118221500219192192522204227392275322869229012322123746239262395224177253062533725392258482646426575270962745827979280572831128440284662847728805295422997330407315373184232396324403248432648328733291833089334143387034100346263489535172353473643536501365233707337306383193888039278396353978340431405584086241212415164170541752421304235042458425344270142747446684505045197453584543845487457834592246597469224729047472480894887649402494184946450326503445042751258521135240453439538385426654313544825478055046554035558955606560815698457092571765725357705578765853758610586995924059418601696036060590608286090161686626486279862984633376341563523637176431064875651336520565522656266571566786673026842468784693166955069683699647008570306706517120672404725207271774015